In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

import re
import time

rm_parenthesis = re.compile(r"\（.*?\）|\（ .*? \）|\(.*?\)|\( .*? \)")

## Получение датасета с пиньинем

In [2]:
options = Options()
options.headless = True
driver = webdriver.Firefox(options=options)

try:
    driver.get("https://mandarinbean.com/new-hsk-1-word-list/")
    headers = [
        header.get_attribute("innerHTML") for header in driver.find_elements(By.TAG_NAME, "strong")]
    final_data = pd.DataFrame(columns=headers)
    temp_data = [
        value.get_attribute("innerHTML") for value in driver.find_elements(By.TAG_NAME, "td")]

except Exception as e:
    print(f"{e}\n\nThe process will terminate in 10 seconds...")
    time.sleep(10)

finally:
    driver.close()
    driver.quit()

In [52]:
m_final_data = final_data
m_temp_data = temp_data

In [54]:
def divide_rows(row):   # Разделяет один ряд с n-ным количеством "|" на n+1 рядов
    result = []
    while "｜" in "".join(row):
        irow = [
            row[0], row[1][:row[1].index("｜")], row[2][:row[2].index("｜")-1], row[3]]
        row = [
            row[0], row[1][row[1].index("｜")+1:], row[2][row[2].index("｜")+2:], row[3]]
        result.append(irow)
    result.append([row[0], row[1][0:], row[2][0:], row[3]])
    return result 

data = []
for i in range(4, len(m_temp_data), 4):
    temp_arr = [
        re.sub(rm_parenthesis, "", td.replace("&nbsp;", " ")) for td in m_temp_data[i-4:i]]  
    if "｜" in "".join(temp_arr):
        temp_arr = divide_rows(temp_arr)
    data.extend([temp_arr] if len(temp_arr)==4 else temp_arr)

m_final_data = pd.DataFrame(data=data, columns=headers)
m_final_data.drop(labels=["No", "English"], axis=1, inplace=True)
m_final_data.head()
m_final_data.to_csv(path_or_buf=".\\test.csv", index=False)

## Скрейпинг для других систем транскрипции

### Палладий

In [11]:
final_data = pd.read_csv("./test.csv")

driver = webdriver.Chrome()

try:
    def convert_system(
        url, 
        src_table, 
        textbox, 
        convert_button, 
        output, 
        system_selector=None, 
        variations=None
        ):

        def clear_text(element):
            length = len(element.get_attribute('value'))
            element.send_keys(length * Keys.BACKSPACE)

        if variations is not None:
            pass

        driver.get(url)       
        scraped = []
        
        target = driver.find_element(By.CLASS_NAME, textbox)
        wait = WebDriverWait(driver, 10)
        wait.until(lambda w: target.is_displayed())

        textbox = driver.find_element(By.CLASS_NAME, textbox)
        convert_button = (By.CLASS_NAME, convert_button)
        output = driver.find_element(By.CLASS_NAME, output)

        for tra in src_table:
            textbox.send_keys(tra)
            convert_button.click()
    
            driver.implicitly_wait(1)
            scraped.append(driver.find_element(output).get_attribute("innerHTML"))
            clear_text(textbox)
        return scraped
    
    palladius = pd.Series(
        data=convert_system(
            "https://palladius.ru/", 
            [tra for tra in final_data["Pinyin"]],
            "pinyin",
            "translate",
            "palladius"
            )
        )
    final_data = pd.concat(objs=[final_data, palladius], axis=1, index=False)
    print(final_data)

except Exception as e:
    print(f"{e}\n\nThe process will terminate in 10 seconds...")
    time.sleep(10)

finally:
    driver.close()
    driver.quit()

Message: invalid argument: 'using' must be a string
  (Session info: chrome=119.0.6045.200)
Stacktrace:
	GetHandleVerifier [0x00007FF600F182B2+55298]
	(No symbol) [0x00007FF600E85E02]
	(No symbol) [0x00007FF600D405AB]
	(No symbol) [0x00007FF600D8165F]
	(No symbol) [0x00007FF600D818DC]
	(No symbol) [0x00007FF600DBCBC7]
	(No symbol) [0x00007FF600DA20EF]
	(No symbol) [0x00007FF600DBAAA4]
	(No symbol) [0x00007FF600DA1E83]
	(No symbol) [0x00007FF600D7670A]
	(No symbol) [0x00007FF600D77964]
	GetHandleVerifier [0x00007FF601290AAB+3694587]
	GetHandleVerifier [0x00007FF6012E728E+4048862]
	GetHandleVerifier [0x00007FF6012DF173+4015811]
	GetHandleVerifier [0x00007FF600FB47D6+695590]
	(No symbol) [0x00007FF600E90CE8]
	(No symbol) [0x00007FF600E8CF34]
	(No symbol) [0x00007FF600E8D062]
	(No symbol) [0x00007FF600E7D3A3]
	BaseThreadInitThunk [0x00007FFD5FAD7344+20]
	RtlUserThreadStart [0x00007FFD5FC426B1+33]


The process will terminate in 10 seconds...


In [4]:
options = Options()
options.add_argument("--headless=new")
driver = webdriver.Chrome(options=options)

try:
    driver.get("https://mandarinbean.com/new-hsk-1-word-list/")
    headers = [header.get_attribute("innerHTML") for header in driver.find_elements(By.TAG_NAME, "strong")]
    final_data = pd.DataFrame(columns=headers)

    def get_table(url):     # Берёт страницу и собирает с неё все элементы таблицы (в тегах <td>) в датафрейм

        def divide_rows(row):   # Разделяет один ряд с n-ным количеством "|" на n+1 рядов
            result = []
            while "｜" in "".join(row):
                irow = [row[0], row[1][:row[1].index("｜")], row[2][:row[2].index("｜")-1], row[3]]
                row = [row[0], row[1][row[1].index("｜")+1:], row[2][row[2].index("｜")+2:], row[3]]
                result.append(irow)
            result.append([row[0], row[1][0:], row[2][0:], row[3]])
            return result 

        driver.get(url)
        element = WebDriverWait(driver, 10).until(          # Ожидание прогрузки элементов в тегах <td>
            EC.presence_of_element_located((By.TAG_NAME, "td"))
        )

        temp_data = [value.get_attribute("innerHTML") for value in driver.find_elements(By.TAG_NAME, "td")]

        data = []
        for i in range(4, len(temp_data), 4):
            temp_arr = [re.sub(rm_parenthesis, "", td.replace("&nbsp;", " ")) for td in temp_data[i-4:i]]
            if "｜" in "".join(temp_arr):
                temp_arr = divide_rows(temp_arr)
            data.extend([temp_arr] if len(temp_arr)==4 else temp_arr)

        return pd.DataFrame(data=data, columns=headers)

    for i in range(1, 7):
        final_data = pd.concat(objs=[final_data, get_table(f"https://mandarinbean.com/new-hsk-{i}-word-list/")]) 
        
    final_data.drop(labels=["No", "English"], axis=1, inplace=True)
    final_data.to_csv(path_or_buf=".\\pinyin.csv", index=False)

except Exception as e:
    print(f"{e}\n\nThe process will terminate in 10 seconds...")
    time.sleep(10)

finally:
    driver.close()
    driver.quit()

In [24]:
data = pd.read_csv("./pinyin.csv")
data.head()

,Chinese,Pinyin,English
0,爱,ài,love
1,爱好,ài hào,hobby
2,八,bā,eight
3,爸爸｜爸,bàba ｜ bà,dad
4,吧,ba,(interjection particle)
